In [1]:
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 6.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [107]:
import sys
!{sys.executable} -m pip install geocoder

In [106]:
!conda install -c conda-forge geopy --yes 


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


### Question 1 + Question 2
I did question 1 and 2 together as it was easier to code. The final dataframe is the result og 

In [136]:
#Import the libraries
import numpy as np 
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

In [93]:
#Access to the data in Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
web = requests.get(url) 
soup = BeautifulSoup(web.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

In [94]:
#Extract and clean the data to put it into a dataframe
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned\n':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                

In [95]:
#Example of a neighborhood
neighborhoods[5]

'Islington Avenue\n'

In [96]:
#Create the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods1 = pd.DataFrame(columns=column_names)
neighborhoods1

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [97]:
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    PostalC = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(PostalC))
    lat_lng_coords = g.latlng

    neighborhoods1 = neighborhoods1.append({ 'PostalCode': PostalC,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighborhoods1

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641
1,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.661790,-79.389390
...,...,...,...,...,...
97,M5X\n,Downtown Toronto\n,"First Canadian Place, Underground city\n",43.648269,-79.381884
98,M8X\n,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n",43.653340,-79.509766
99,M4Y\n,Downtown Toronto\n,Church and Wellesley\n,43.666659,-79.381472
100,M7Y\n,East Toronto\n,Business reply mail Processing Centre\n,43.648700,-79.385450


### Question 3
To do the clustering of the different neighborhoods I sued the same method than  in the NY example

In [110]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [115]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(neighborhoods1['Latitude'], neighborhoods1['Longitude'], neighborhoods1['Borough'],neighborhoods1['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [116]:
CLIENT_ID = 'S0PWMD5GYQFMEIPWXSKV30JCD41E2N2E0SNDIDOMMIJSPRKM' # your Foursquare ID
CLIENT_SECRET = 'TEOZAZDTLH05ZZ4M1LRM4T0LCO2ZZCFSGOX4AQIPXKZY5KKQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S0PWMD5GYQFMEIPWXSKV30JCD41E2N2E0SNDIDOMMIJSPRKM
CLIENT_SECRET:TEOZAZDTLH05ZZ4M1LRM4T0LCO2ZZCFSGOX4AQIPXKZY5KKQ


In [119]:
neighborhoods1.loc[0, 'Neighborhood']
neighborhood_latitude = neighborhoods1.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods1.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods1.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods
 are 43.75293455500008, -79.33564142299997.


In [120]:
LIMIT =100
radius=500
URL=  'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
URL


'https://api.foursquare.com/v2/venues/explore?&client_id=S0PWMD5GYQFMEIPWXSKV30JCD41E2N2E0SNDIDOMMIJSPRKM&client_secret=TEOZAZDTLH05ZZ4M1LRM4T0LCO2ZZCFSGOX4AQIPXKZY5KKQ&v=20180605&ll=43.75293455500008,-79.33564142299997&radius=500&limit=100'

In [123]:
results = requests.get(URL).json()
results

{'meta': {'code': 200, 'requestId': '5eb81588aba297001c1ae5e0'},
 'response': {'headerLocation': 'Sunnybrook - York Mills',
  'headerFullLocation': 'Sunnybrook - York Mills, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.75743455950008,
    'lng': -79.32942319651914},
   'sw': {'lat': 43.74843455050008, 'lng': -79.3418596494808}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 301,
        

In [124]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [127]:
Toronto_venues = getNearbyVenues(names=neighborhoods1['Neighborhood'],
                                   latitudes=neighborhoods1['Latitude'],
                                   longitudes=neighborhoods1['Longitude'])


Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government

Islington Avenue

Malvern, Rouge

Don Mills

Parkview Hill, Woodbine Gardens

Garden District, Ryerson

Glencairn

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale

Rouge Hill, Port Union, Highland Creek

Don Mills

Woodbine Heights

St. James Town

Humewood-Cedarvale

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Guildwood, Morningside, West Hill

The Beaches

Berczy Park

Caledonia-Fairbanks

Woburn

Leaside

Central Bay Street

Christie

Cedarbrae

Hillcrest Village

Bathurst Manor, Wilson Heights, Downsview North

Thorncliffe Park

Richmond, Adelaide, King

Dufferin, Dovercourt Village

Scarborough Village

Fairview, Henry Farm, Oriole

Northwood Park, York University

East Toronto

Harbourfront East, Union Station, Toronto Islands

Little Portugal, Trinity

Kennedy Park, Ionview, East Birchmount Park

Bayview 

In [128]:
Toronto_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods\n,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods\n,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods\n,43.752935,-79.335641,Three Valleys Park,43.751195,-79.337356,Park
3,Parkwoods\n,43.752935,-79.335641,KFC,43.754387,-79.333021,Fast Food Restaurant
4,Victoria Village\n,43.728102,-79.311890,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [131]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Agincourt\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch\n",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North\n",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0
3,Bayview Village\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East\n",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


We want to find the most common 5 venues for each Neighborhood

In [133]:

num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0     Breakfast Spot  0.25
1        Supermarket  0.25
2    Badminton Court  0.25
3   Sushi Restaurant  0.25
4  Mobile Phone Shop  0.00


----Alderwood, Long Branch
----
               venue  freq
0     Sandwich Place   0.1
1        Pizza Place   0.1
2  Convenience Store   0.1
3           Pharmacy   0.1
4       Dance Studio   0.1


----Bathurst Manor, Wilson Heights, Downsview North
----
           venue  freq
0           Bank  0.11
1    Coffee Shop  0.11
2    Bridal Shop  0.05
3       Pharmacy  0.05
4  Deli / Bodega  0.05


----Bayview Village
----
                        venue  freq
0                       Trail   0.5
1  Construction & Landscaping   0.5
2                 Yoga Studio   0.0
3     New American Restaurant   0.0
4                 Music Venue   0.0


----Bedford Park, Lawrence Manor East
----
                venue  freq
0  Italian Restaurant  0.10
1      Sandwich Place  0.10
2         Coffee Shop  0.10
3          Restaurant  0.0

In [134]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [135]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt\n,Breakfast Spot,Sushi Restaurant,Badminton Court,Supermarket,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Dog Run
1,"Alderwood, Long Branch\n",Athletics & Sports,Gym,Sandwich Place,Dance Studio,Pub,Coffee Shop,Gas Station,Convenience Store,Pharmacy,Pizza Place
2,"Bathurst Manor, Wilson Heights, Downsview North\n",Coffee Shop,Bank,Pizza Place,Ice Cream Shop,Shopping Mall,Middle Eastern Restaurant,Sandwich Place,Fried Chicken Joint,Sushi Restaurant,Restaurant
3,Bayview Village\n,Trail,Construction & Landscaping,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Women's Store
4,"Bedford Park, Lawrence Manor East\n",Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Juice Bar,Café,Butcher,Pub,Sports Club,Indian Restaurant


In [137]:
kclusters = 6
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([3, 3, 3, 4, 3, 3, 3, 3, 3, 3], dtype=int32)

In [138]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = neighborhoods1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641,0.0,Park,Food & Drink Shop,Fast Food Restaurant,Distribution Center,Fish Market,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant
1,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890,3.0,Pizza Place,Park,Intersection,French Restaurant,Portuguese Restaurant,Coffee Shop,Falafel Restaurant,Farm,Ethiopian Restaurant,Diner
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041,3.0,Pub,Café,Athletics & Sports,Coffee Shop,Optical Shop,Tech Startup,Mediterranean Restaurant,Mexican Restaurant,Food Truck,Restaurant
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211,3.0,Clothing Store,Pharmacy,Food Court,Cosmetics Shop,Bookstore,Men's Store,Toy / Game Store,American Restaurant,Furniture / Home Store,Restaurant
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.661790,-79.389390,3.0,Coffee Shop,Café,Sushi Restaurant,Hobby Shop,Fried Chicken Joint,Restaurant,Pharmacy,Park,Middle Eastern Restaurant,Juice Bar


In [141]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters